In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

/home/visionnoob/anaconda3/envs/py36tf17/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
np.random.seed(42)

In [3]:
from keras.applications.vgg16 import preprocess_input

train_datagen = ImageDataGenerator(
    #rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split = 0.2)

#train_datagen = ImageDataGenerator(rescale=1./255, validation_split = 0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

train_datagen = ImageDataGenerator(validation_split = 0.013, preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
#categorical
#binary
train_generator = train_datagen.flow_from_directory(
        'small_dataset/train',
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical',
        subset = 'training')

vali_generator = train_datagen.flow_from_directory(
        'small_dataset/train',
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical',
        subset = 'validation')

test_generator = test_datagen.flow_from_directory(
        'small_dataset/test',
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical')

Found 2568 images belonging to 2 classes.
Found 32 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


# MAIN

In [4]:
from keras.applications import VGG16


conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [5]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [11]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
#model.add(layers.Dense(2, activation = 'softmax'))
model.add(layers.Dense(2, activation = 'softmax'))

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 514       
Total params: 16,812,610
Trainable params: 2,097,922
Non-trainable params: 14,714,688
_________________________________________________________________


In [13]:
print('This is the number of trainable weights '
      'before freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights before freezing the conv base: 4


In [14]:
conv_base.trainable = False

In [15]:
print('This is the number of trainable weights '
      'after freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights after freezing the conv base: 4


In [16]:
import keras
model.compile(loss='categorical_crossentropy',optimizer='adam',
              metrics=['acc'])
#categorical_crossentropy
#binary_crossentropy
#mySGD = keras.optimizers.SGD(lr=0.1, momentum=0.99, decay=0.0, nesterov=False)

#model.compile(loss='categorical_crossentropy',optimizer=mySGD,
  #            metrics=['acc'])
      

In [ ]:
import keras
model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

In [ ]:
import keras
callback_func = keras.callbacks.RemoteMonitor(root='http://localhost:9000', path='/publish/epoch/end/', field='data', headers=None, send_as_json=False)
tb_hist = keras.callbacks.TensorBoard(log_dir='./graph', histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=40,
        epochs=20, callbacks =[callback_func])

In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=30, 
        validation_data = vali_generator,
        validation_steps=50, callbacks =[callback_func, tb_hist])

In [17]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=30)

Epoch 1/30
100/100 [==============================] - 16s 159ms/step - loss: 4.9517 - acc: 0.6883
Epoch 2/30
100/100 [==============================] - 14s 142ms/step - loss: 2.2072 - acc: 0.8612
Epoch 3/30
100/100 [==============================] - 14s 143ms/step - loss: 1.5897 - acc: 0.9000
Epoch 4/30
100/100 [==============================] - 15s 146ms/step - loss: 1.8887 - acc: 0.8823
Epoch 5/30
100/100 [==============================] - 14s 141ms/step - loss: 1.8402 - acc: 0.8853
Epoch 6/30
100/100 [==============================] - 14s 142ms/step - loss: 1.5169 - acc: 0.9048
Epoch 7/30
100/100 [==============================] - 14s 144ms/step - loss: 1.7080 - acc: 0.8930
Epoch 8/30
100/100 [==============================] - 14s 139ms/step - loss: 1.5705 - acc: 0.9020
Epoch 9/30
100/100 [==============================] - 14s 145ms/step - loss: 1.7469 - acc: 0.8915
Epoch 10/30
100/100 [==============================] - 14s 142ms/step - loss: 1.7852 - acc: 0.8880
Epoch 11/30
100/100

# INFERENCE

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights='imagenet', include_top=True)

In [ ]:
MEAN_VALUE = np.array([103.939, 116.779, 123.68])   # BGR
def preprocess(img):
    # img is (channels, height, width), values are 0-255
    img = img[...,::-1]  # switch to BGR
    img -= MEAN_VALUE
    return img

In [ ]:
img_path = 'n02088094_7.JPEG'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess(x)
#x = preprocess_input(x)
yhat = model.predict(x)
print(x[0][0][0])

In [ ]:
from keras.applications.vgg16 import decode_predictions
# convert the probabilities to class labels
label = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)